# Chapter 5: Responsible Prompting

---

**Lesson:**

While there is enormous potential with Generative AI models, we must always take into account the inherent risks of these models, particularly as it pertains to the effect on people / groups of people.

In [ ]:
%%capture
# Install dependencies
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet "pillow>=9.5,<10"

# Python Built-Ins:
import base64
import io
import json
import os
import sys

# External Dependencies:
import boto3
from PIL import Image

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

modelId = "stability.stable-diffusion-xl"

## The inherent risk of our models

Stable Diffusion and all other image generation models are subject to some level of risk, whether it is due to malicious or non-malicious intent. Let's discuss two of the most pertinent topics for these models.

### Malicious image production

Not everyone has positive intent when developing images via Stable Diffusion. Misue of models can include efforts to defame or dehumanize people or groups of people. These efforts can range from ad hominem attacks in a community to political statements (particulalrly during elections) and attacks on cultures and beliefs. Model providers and the wider community continue to innovate to prevent the intentional misuse of models. However, it is very important to keep in mind the potential for misuse both in models you deploy as well as for images you see day to day that may have been modified to demean a person or people. As a user of SDXL, continually keep up to date with any "holes" in the model's ability to detect and deny malicious intent, and work to prevent misuse by any users of your application.

### Attribute association

An unfortunate side effect of many models is assumed correlation of certain attributes (such as eye and hair color) with other attributes (such as race). While this attribute association represents non-malicious intent by the model, the effect is similar to the above - poor representation of certain segments of the population. What's more is this issue can spill into other areas beyond race, culture, etc. as our models associate keywords incorrectly and generate undesired images (e.g., sustainability is most often associated with green technologies / ways of living, but this term can also simply designate a long-term, strategic focus). With this in mind, it is important to pay attention to the keywords you use and understand if your model will incorrectly assume what you are looking for with the word(s) in your prompt.

Let's explore this concept in a couple of examples.

**Example 5.1 - Blue eyes**

While there are certainly populations that predominantly have blue eyes, this eye color is not limited solely to those groups of people. However, as Stable Diffusion makes decisions based on probabilities, the model will often generate based on what it has "seen" most often in its original, training corpus of online data.

In [ ]:
prompt = "a person with blue eyes"

In [ ]:
request = json.dumps({
    "text_prompts": (
        [{"text": prompt}]
    ),
})

response = boto3_bedrock.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

print(response_body["result"])
base_64_img_str = response_body["artifacts"][0].get("base64")
print(f"{base_64_img_str[0:80]}...")

os.makedirs("data", exist_ok=True)
eg5_1 = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
eg5_1.save("data/eg5_1.png")
eg5_1

**Example 5.2 - What is football?**

Let's see if SDXL recognizes football can mean different things depending on where you are in the world.

In [ ]:
prompt = "football"

In [ ]:
request = json.dumps({
    "text_prompts": (
        [{"text": prompt}]
    ),
})

response = boto3_bedrock.invoke_model(body=request, modelId=modelId)
response_body = json.loads(response.get("body").read())

print(response_body["result"])
base_64_img_str = response_body["artifacts"][0].get("base64")
print(f"{base_64_img_str[0:80]}...")

os.makedirs("data", exist_ok=True)
eg5_2 = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
eg5_2.save("data/eg5_2.png")
eg5_2

One of the best fixes to the above issues is adding more keywords, clarifying what you are looking for. However, it is very important to understand the inherent biases beneath the surface of your models, at the very least, so you can understand why you may not be getting the results you desire and how you can modify your prompts to produce more accurate results.